In [13]:

import torch
import torch
import numpy as np
import sys
import os
import general
sys.path.append(os.path.join(os.path.abspath(''), 'core'))
from zeropi import ZeroPi
import general 
import utils as utl



In [24]:
#Gradient Descent over T2 rate

#FIXED PARAMS
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
ECS= torch.tensor(0.01, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = 0.5
ng = 0.1
ncut = 30
hamiltonian_creation_solution = "manual_discretization_davidson"
discretization_dim = 100
learn_rate =10

#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EL_bounds = [2e-1, 10]

#RECORDING VALUES
EJ_vals = [EJ.item()]
EL_vals = [EL.item()]
T2_rate_results = [1e100, 1e99]

#we are trying to minise the rate

#GRADIENT DESCENT
i = 0
for x in range(0,10) :
    i+=1

    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)

    zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ,
                 ECS = ECS, 
                 dCJ=dCJ, 
                 flux = flux,
                 ng=ng, 
                 ncut = ncut,
                 discretization_dim =discretization_dim,
                 hamiltonian_creation_solution=hamiltonian_creation_solution, 
                 sparse = True)
    
    eigvals, eigvecs = zeropi.esys()

    T2_rate  = general.t2_rate(qubit = zeropi, eigvecs=eigvecs, eigvals =eigvals)

    T2_rate.backward()
    
    T2_rate_results.append(T2_rate.item())

    with torch.no_grad():
        #If paramater value hits boundary, it stays on boundary
        ##WHY DO WE minus the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EL_vals.append(EL.item())

    print(f"{i}: R2: {T2_rate.item()}, EJ: {EJ.item()}, EL: {EL.item()}")



1: R2: 0.00028234018993202666, EJ: 14.44843409982512, EL: 8.865663016611212
2: R2: 0.00028234018993197456, EJ: 14.44843409982594, EL: 8.865663016590858
3: R2: 0.00028234018993192426, EJ: 14.44843409982676, EL: 8.865663016570505
4: R2: 0.0002823401899318722, EJ: 14.448434099827582, EL: 8.865663016550151
5: R2: 0.00028234018993181985, EJ: 14.448434099828402, EL: 8.865663016529798
6: R2: 0.0002823401899317677, EJ: 14.448434099829223, EL: 8.865663016509444
7: R2: 0.000282340189931711, EJ: 14.448434099830044, EL: 8.86566301648909
8: R2: 0.00028234018993166416, EJ: 14.448434099830864, EL: 8.865663016468737
9: R2: 0.0002823401899316089, EJ: 14.448434099831685, EL: 8.865663016448384
10: R2: 0.00028234018993155595, EJ: 14.448434099832506, EL: 8.86566301642803


In [27]:
#Gradient Descent Over Protection Factor

t1_noise_channels =  zeropi.t1_supported_noise_channels()
tphi_noise_channels =  zeropi.tphi_supported_noise_channels()

#FIXED PARAMS
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
ECS= torch.tensor(0.01, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = 0.5
ng = 0.1
ncut = 30
hamiltonian_creation_solution = "manual_discretization_davidson"
discretization_dim = 100
learn_rate =1


#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EL_bounds = [2e-1, 10]

delta_EJ = ((EJ_bounds[1]-EJ_bounds[0])/2)*0.1
delta_EL = ((EL_bounds[1]-EL_bounds[0])/2)*0.1

#RECORDING VALUES
EJ_vals = [EJ.item()]
EL_vals = [EL.item()]
protection_factor_results = [1e100, 1e99]

#we are trying to minise the T2 rate

#GRADIENT DESCENT
i = 0
for x in range(0,10):
    i+=1

    #Initiate qubit with current paramater values
    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)

    zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ,
                 ECS = ECS, 
                 dCJ=dCJ, 
                 flux = flux,
                 ng=ng, 
                 ncut = ncut,
                 discretization_dim =discretization_dim,
                 hamiltonian_creation_solution=hamiltonian_creation_solution, 
                 sparse = True)
    
    eigvals, eigvecs = zeropi.esys()

    T1_rate  = general.effective_t1_rate(qubit = zeropi, eigvecs=eigvecs, eigvals =eigvals, noise_channels=t1_noise_channels)

    Tphi_rate  = general.effective_tphi_rate(qubit = zeropi, eigvecs=eigvecs,  noise_channels=tphi_noise_channels)


    #Initiate qubit with current paramater values + delta
    EJ = torch.tensor(EJ.item()+delta_EJ, requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item()+delta_EL, requires_grad=True, dtype=torch.double)
    zeropi_delta = ZeroPi(EJ=EJ, 
                EL=EL , 
                ECJ=ECJ, 
                EC=EC, 
                dEJ=dEJ,
                ECS = ECS, 
                dCJ=dCJ, 
                flux = flux,
                ng=ng, 
                ncut = ncut,
                discretization_dim =discretization_dim,
                hamiltonian_creation_solution=hamiltonian_creation_solution, 
                sparse = True)

    eigvals, eigvecs = zeropi_delta.esys()

    T1_rate_delta  = general.effective_t1_rate(qubit = zeropi_delta, eigvecs=eigvecs, eigvals =eigvals, noise_channels=t1_noise_channels)

    Tphi_rate_delta  = general.effective_tphi_rate(qubit = zeropi_delta, eigvecs=eigvecs, noise_channels=tphi_noise_channels)

    #Calculate protection factor
    protection_factor = 0.5*( (T1_rate - T1_rate_delta)/(Tphi_rate - Tphi_rate_delta) + (Tphi_rate - Tphi_rate_delta)/(T1_rate - T1_rate_delta)) 

    #Calculate gradients and apply GD
    protection_factor.backward()
    
    protection_factor_results.append(protection_factor.item())

    with torch.no_grad():
        #If paramater value hits boundary, it stays on boundary
        ##WHY DO WE minus the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
 
    EL_vals.append(EL.item())

    print(f"{i}: Protection Factor: {protection_factor.item()}, EJ: {EJ.item()}, EL: {EL.item()}")



1: R2: 1229290.8870898143, EJ: 2.5, EL: 269855.3586696291
2: R2: 7100768357.860662, EJ: 8.239557624647853, EL: 1932566.6507599277
3: R2: nan, EJ: 2.5, EL: 0.20000000298023224
